In [13]:
import requests
import config
import json
import os

GOOGLE_API_KEY = config.google_api_key
SUDO_PASSWORD = config.sudo_pwd

In [21]:
# These are all util functions, not part of main script

def addRestaurants(restaurants):
    for name, address in restaurants:
        loc = geocode(address)
        savedRestaurants[name] = loc
    storeRestaurants()

def geocode(address):
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    params = {'address': address, 'key': GOOGLE_API_KEY}
    r = requests.get(url, params=params)
    results = r.json()['results']
    location = results[0]['geometry']['location']
    return location['lat'], location['lng']

def currLocation():
    url = "https://www.googleapis.com/geolocation/v1/geolocate?key=" + GOOGLE_API_KEY
    response = requests.post(url)
    location_data = response.json()['location']
    lat, long = location_data['lat'], location_data['lng']
    return ((float(lat), float(long)))

def distance(loc1, loc2):
    x, y = loc1
    X, Y = loc2
    
    a, b = X-x, Y-y
    
    return (a**2 + b**2)**1/2

def sortByDistance():
    currLoc = currLocation()
    sortedRestaurants = sorted(savedRestaurants.items(), key=lambda data: distance(currLoc, data[1]))
    
    return [name for name, loc in sortedRestaurants]

In [22]:
currLocation()

(47.6204358, -122.2069924)

In [23]:
def loadRestaurants():
    try:
        with open('data/restaurants') as restaurants_file:
            global savedRestaurants
            savedRestaurants = json.load(restaurants_file)
    except IOError:
        print("Error loading restaurants")

def storeRestaurants():
    with open('data/restaurants', 'w') as restaurants_file: 
        json.dump(savedRestaurants, restaurants_file)

In [24]:
# TODO: load from database
savedRestaurants = {}
loadRestaurants()

In [25]:
savedRestaurants

{'Uwajimaya': [47.61584680000001, -122.1824689],
 'Fred Meyer': [47.6284792, -122.1437256],
 'Bellevue Square': [47.6156024, -122.203819],
 'The Home Depot': [47.6124246, -122.181756]}

In [26]:
toAdd = [('Uwajimaya', "699 120th Ave NE, Bellevue, WA 98005"), 
         ('Fred Meyer', "2041 148th Ave NE, Bellevue, WA 98007"),
        ('Bellevue Square', "575 Bellevue Square, Bellevue, WA 98004"),
        ("The Home Depot", "325 120th Ave NE, Bellevue, WA 98005")]
addRestaurants(toAdd)

In [27]:
sortByDistance()

['Bellevue Square', 'Uwajimaya', 'The Home Depot', 'Fred Meyer']

In [ ]:
t